In [1]:
!pip install flair PyMuPDF spacy openpyxl pandas nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 15.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 76.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.6/202.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!python -m spacy download es_core_news_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 11.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [10]:
import fitz  # PyMuPDF
import re
import nltk
import pandas as pd
import spacy
from flair.data import Sentence
from flair.models import SequenceTagger
from collections import Counter
from nltk.corpus import stopwords
from huggingface_hub import login

In [12]:
# Introduce tu token aquí
token = "xxxxxxxxxxxx"

login(token=token)

# Ahora puedes cargar el modelo
# The model name has been changed from 'ner-spanish' to 'flair/ner-spanish-large'
tagger = SequenceTagger.load('flair/ner-spanish-large')

pytorch_model.bin:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

2025-02-14 16:33:23,271 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-LOC, S-ORG, B-PER, I-PER, E-PER, S-MISC, B-ORG, E-ORG, S-PER, I-ORG, B-LOC, E-LOC, B-MISC, E-MISC, I-MISC, I-LOC, <START>, <STOP>


In [13]:
# Ruta del archivo PDF
pdf_path = '/content/certificado (2).pdf'

# 1. Leer y unificar el texto
def read_pdf(file_path):
    doc = fitz.open(file_path)
    text = ''
    for page in doc:
        text += page.get_text()
    doc.close()
    return text

In [14]:
# 2. Preprocesamiento del texto
def preprocess_text(text):
    # Unificar Unicode
    text = text.encode('utf-8').decode('utf-8')
    # Eliminar caracteres especiales y números
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'\d+', '', text)
    # Convertir a minúsculas
    text = text.lower()
    # Tokenizar, eliminar stopwords y lematizar
    stop_words = set(stopwords.words('spanish'))
    tokens = [token.lemma_ for token in nlp(text) if token.text not in stop_words and token.is_alpha]
    return ' '.join(tokens)

In [15]:
# 3. Aplicar NER con Flair
def apply_ner(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = [(entity.text, entity.labels[0].value) for entity in sentence.get_spans('ner')]
    return entities


In [16]:
# 4. Contar frecuencia de entidades
def count_entities(entities):
    counter = Counter(entities)
    data = [{'Entidad': ent, 'Tipo': typ, 'Frecuencia': freq} for (ent, typ), freq in counter.items()]
    df = pd.DataFrame(data)
    return df

In [17]:
# 5. Guardar resultados en Excel
def save_to_excel(df, output_path='/content/NER_Frecuencias.xlsx'):
    df.to_excel(output_path, index=False)
    print(f'Resultados guardados en: {output_path}')

In [18]:
# Pipeline completo
text = read_pdf(pdf_path)
clean_text = preprocess_text(text)
entities = apply_ner(clean_text)
result_df = count_entities(entities)

print(result_df)
save_to_excel(result_df)

                  Entidad  Tipo  Frecuencia
0       wwwcamarabaqorgco  MISC           1
1         sandoval orozco   PER           2
2  barranquillo atlantico   LOC           2
3                colombia   LOC           1
4    sandoval abdo wilmis   PER           1
5     ardilar madin jesus   PER           1
Resultados guardados en: /content/NER_Frecuencias.xlsx
